In [1]:
#CONSULTA PARA REALIZAR LOS POTENCIALES, SON DOS CELDAS A EJECUTAR -SE PUEDE EJECUTAR LA CELDA 1 Y 2, O LA CELDA 1 Y 3. 
#LA PRIMER CELDA ES LA CONEXION A LA BASE DE DATOS
#LA CELDA2 NO TE DIVIDE POR ACTIVO- EN CAMBIO LA CELDA 3 SI. 
#LA CELDA CUATRO- EXPORTA LOS ARCHIVOS .CSV DIRECTAMENTE PARA CARGARLOS A TOW - SIEMPRE Y CUANDO SE HAYA EJECUTADO LA CELDA 1,3 Y 4

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
from sqlalchemy import create_engine

def main():
    # Parámetros de conexión Oracle (Easy Connect)
    usuario  = "RY16123"
    password = "Luciano281"
    host     = "suarbuworap09"
    puerto   = 1527
    servicio = "psfu"

    # Cadena de conexión usando Easy Connect
    conn_str = f"oracle+cx_oracle://{usuario}:{password}@{host}:{puerto}/{servicio}"
    engine   = create_engine(conn_str)

    # Consulta a ejecutar
    sql = """
    SELECT MONTHS_BETWEEN(SYSDATE,VTOW_WELL_LAST_POTENTIAL_DET.MAX_EFF_DT), MONTHS_BETWEEN(SYSDATE,FIC_ULT_DIAGNOSTICO.FECHA_DESDE)*30, SYSDATE-VTOW_WELL_LAST_POTENTIAL_DET.MAX_EFF_DT, DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO, DBU_FIC_ORG_ESTRUCTURAL.ESTADO, DBU_FIC_ORG_ESTRUCTURAL.MET_PROD, DBU_FIC_ORG_ESTRUCTURAL.NIVEL_1, DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3, DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5, TEST_FIC_ULT_NIVELES.FECHA_NVL, VTOW_WELL_LAST_CONTROL_DET.MAX_EFF_DT, VTOW_WELL_LAST_CONTROL_DET.TEST_PURP_CD, VTOW_WELL_LAST_CONTROL_DET.TEST_REASON_CD, VTOW_WELL_LAST_POTENTIAL_DET.MAX_EFF_DT, TEST_FIC_ULT_NIVELES.TIPONIVEL, DBU_FIC_ORG_ESTRUCTURAL.CLAS_ABC, FIC_ULT_DIAGNOSTICO.FECHA_DESDE, FIC_ULT_DIAGNOSTICO.DIAGNOSTICO, FIC_ULT_DIAGNOSTICO.MOTIVO, ( VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT )-VTOW_WELL_LAST_CONTROL_DET.TOTAL_VOL, VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL-VTOW_WELL_LAST_CONTROL_DET.PROD_OIL, DBU_FIC_ORG_ESTRUCTURAL.COMP_SK, VTOW_WELL_LAST_CONTROL_DET.DAYS_FROM_TODAY, VTOW_WELL_LAST_CONTROL_DET.PROD_OIL, VTOW_WELL_LAST_CONTROL_DET.PROD_GAS, VTOW_WELL_LAST_CONTROL_DET.FREE_WAT_PCT, VTOW_WELL_LAST_CONTROL_DET.TOTAL_VOL, VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL, VTOW_WELL_LAST_POTENTIAL_DET.POT_GAS, ( VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT*100 )/DECODE(( VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT ),0,1,NULL,1,( VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT )), VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT, TEST_FIC_ULT_NIVELES.SUMERGENCIA, TEST_FIC_ULT_NIVELES.REGIMEN_PRIM,TEST_FIC_ULT_NIVELES.PROF_ENT_BOMBA, TEST_FIC_ULT_NIVELES.REGIMEN_SEC
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL, DISC_ADMINS.TEST_FIC_ULT_NIVELES TEST_FIC_ULT_NIVELES, DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET, DISC_ADMINS.VTOW_WELL_LAST_POTENTIAL_DET VTOW_WELL_LAST_POTENTIAL_DET, DISC_ADMINS.FIC_ULT_DIAGNOSTICO FIC_ULT_DIAGNOSTICO
WHERE ( ( VTOW_WELL_LAST_POTENTIAL_DET.COMP_SK(+) = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK ) AND ( VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK ) AND ( TEST_FIC_ULT_NIVELES.COMP_SK(+) = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK ) AND ( DBU_FIC_ORG_ESTRUCTURAL.COMP_SK = FIC_ULT_DIAGNOSTICO.COMP_SK ) ) AND ( DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3 IN ( 'Las Heras CG - Canadon Escondida','Los Perales','El Guadal', 'Seco Leon - Pico Truncado') ) AND ( DBU_FIC_ORG_ESTRUCTURAL.MET_PROD IN ('Auto Gas Lift','Bombeo Hidráulico','Bombeo Mecánico','Cavidad Progresiva','Electro Sumergible','Pistoneo','Plunger Lift','Recoil','Surgente') )
ORDER BY DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5 ASC, DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO ASC

    """

    # Ejecutar y traer el resultado a un DataFrame
    df = pd.read_sql_query(sql, engine)

    # Exportar a Excel
    output_file = "estructura_pozos.xlsx"
    df.to_excel(output_file, index=False)
    print(f"Se exportaron {len(df)} registros a '{output_file}'")

if __name__ == "__main__":
    main()

Se exportaron 6428 registros a 'estructura_pozos.xlsx'


In [6]:
#codigo correcto, este tenemos que usar para calculo de potenciales, sumado el de la consulta ( en el primer renglon)

import pandas as pd
import re
from datetime import datetime
import pandas as pd

# 0) Carga el Excel resultante de tu consulta
df = pd.read_excel("estructura_pozos.xlsx")

# 1) Renombrar columnas
rename_map = {
    'SYSDATE-VTOW_WELL_LAST_POTENTIAL_DET.MAX_EFF_DT':       'Días con potencial',
    'nivel_3':                                                'Activo',
    'nivel_5':                                                'Bateria',
    'prod_oil':                                               'neta',
    'prod_gas':                                               'gas',
    'free_wat_pct':                                           '%H20 control',
    'total_vol':                                              'Bruta',
    'pot_oil':                                                'Neta Potencial',
    'pot_gas':                                                'Gas Potencial',
    'VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT': 
                                                              'Bruta potencial',
    '(VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT*100)/DECODE((VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT),0,1,NULL,1,(VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT))': 
                                                              '%H20 potencial',
    'prof_ent_bomba':                                         'Prof de Bba',
    'sumergencia':                                            'Sumergencia',
    'tiponivel':                                              'Tipo-Nivel',
    'estado':                                                 'Estado',
    'nombre_corto':                                           'Pozo'
}

df = df.rename(columns=rename_map)

# 2) Quedarse solo con esas columnas
df = df[list(rename_map.values())]

# 2.1) Filtrar sólo registros en producción
df = df[df['Estado'] == 'Produciendo']

# 3) Columnas ya existentes calculadas previamente
df['agua'] = df['Bruta'] - df['neta']
df['agua potencial'] = df['Bruta potencial'] - df['Neta Potencial']

# —————————————————————————————————————————————
# 4) Nuevas columnas según tus puntos:
# 4.1) SUM DESEADA: igual a Sumergencia salvo que si >200 vale 200
df['SUM DESEADA'] = df['Sumergencia'].where(df['Sumergencia'] < 200, 200)

# 4.2) P estática: Prof de Bba dividido 10
df['P estática'] = df['Prof de Bba'] / 10

# 4.3) Q máximo: Q_max = Bruta / [1 – 0.2*(Sumergencia/Prof de Bba) – 0.8*(Sumergencia/Prof de Bba)^2]
ratio = df['Sumergencia'] / df['Prof de Bba']
df['Q máximo'] = df['Bruta'] / (1 - 0.2 * ratio - 0.8 * ratio**2)

# 4.4) Q sum deseada: Q_sum = Q_max * [ (1 – 0.2*((SUM DESEADA/10)/P estática)) – 0.8*((SUM DESEADA/10)/P estática)^2 ]
ratio_deseada = (df['SUM DESEADA'] / 10) / df['P estática']
df['Q sum deseada'] = df['Q máximo'] * ((1 - 0.2 * ratio_deseada) - 0.8 * ratio_deseada**2)

# 4.5) Potencial Neta: Q sum deseada * %H20 control
df['Potencial Neta'] = df['Q sum deseada'] * (100 - df['%H20 control']) / 100

# Nuevas columnas de diferencia
# 7) DIF NETA POTENCIAL VS CONTROL
df['DIF NETA POTENCIAL VS CONTROL'] = (df['Potencial Neta'] - df['neta']).round(2)

# 8) % DIF NETA POT VS CONTR (entero nullable para aceptar NaN)
pct = (df['Potencial Neta'] - df['neta']) / df['Potencial Neta'] * 100
df['% DIF NETA POT VS CONTR'] = pct.round(0).astype('Int64')

import numpy as np

# — Inicializo las nuevas columnas —
nuevas = [
    'Potencial Neta Nuevo',
    'Potencial bruta',
    'Nuevo Potencia gas Nuevo',
    '% agua NUEVO potencial',
    'OBSERVACIONES'
]
for col in nuevas:
    df[col] = pd.NA

import re

# — tras tus cálculos previos, antes del guardado —  


# … tu código de carga, rename_map y cálculos previos …

def calcular_nuevas_columnas(row):
    bateria     = str(row['Bateria'])
    sumergencia = row['Sumergencia']
    prof        = row['Prof de Bba']
    neta        = row['neta']
    bruta       = row['Bruta']
    n_pot       = row['Neta Potencial']
    bruta_pot   = row['Bruta potencial']
    gas         = row['gas']
    gas_pot     = row['Gas Potencial']
    pct_ctrl    = row['%H20 control']
    pct_pot     = row['%H20 potencial']
    dif_pct     = row['% DIF NETA POT VS CONTR']
    tipo_nivel  = str(row['Tipo-Nivel']).strip().lower()

    # 0) Swabing / Swabbing
    if re.search(r'swab(?:ing|bing)', bateria, flags=re.IGNORECASE):
        potencia_nueva = neta;  bruta_nueva = bruta
        gas_nuevo      = gas;   pct_nuevo = pct_ctrl
        obs = ''

    # 1) Estático o vacío
    elif tipo_nivel == 'estático' or tipo_nivel == '':
        potencia_nueva = n_pot;    bruta_nueva = bruta_pot
        gas_nuevo      = gas_pot;  pct_nuevo   = pct_pot
        obs = 'ULTIMO VALOR DE SUMERGENCIA ES ESTATICO – SE REPITE POTENCIAL'

    # 2) Sumergencia > Prof de Bba
    elif pd.notna(sumergencia) and sumergencia > prof:
        if neta < n_pot:
            potencia_nueva = n_pot;   bruta_nueva = bruta_pot
            gas_nuevo      = gas_pot; pct_nuevo   = pct_pot
            obs = 'VALOR DE SUMERGENCIA ERRONEO – SE REPITE POTENCIAL'
        else:
            potencia_nueva = neta;    bruta_nueva = bruta
            gas_nuevo      = gas;     pct_nuevo   = pct_ctrl
            obs = 'VALOR DE SUMERGENCIA ERRONEO – SE TOMA COMO POTENCIAL CONTROL'

    # 3) % DIF entre 0 y 30
    elif pd.notna(dif_pct) and 0 <= dif_pct <= 30:
        potencia_nueva = row['Potencial Neta']
        bruta_nueva    = row['Q sum deseada']
        gas_nuevo      = gas
        pct_nuevo      = pct_ctrl
        obs = 'SE CARGA NUEVO POTENCIAL'

    # 4) % DIF entre 30 y 65
    elif pd.notna(dif_pct) and 30 < dif_pct <= 65:
        potencia_nueva = 'DEFINIR INGENIERO'
        bruta_nueva    = 'DEFINIR INGENIERO'
        gas_nuevo      = 'DEFINIR INGENIERO'
        pct_nuevo      = 'DEFINIR INGENIERO'
        obs = 'VALIDAR INGENIERO'

    # 5) % DIF entre 65 y 100
    elif pd.notna(dif_pct) and 65 < dif_pct <= 100:
        potencia_nueva = row['Neta Potencial']
        bruta_nueva    = bruta_pot
        gas_nuevo      = gas_pot
        pct_nuevo      = pct_pot
        obs = 'VER IPR – SE REPITE POTENCIAL'

    # 6) Fallback
    else:
        potencia_nueva = row['Neta Potencial']
        bruta_nueva    = bruta_pot
        gas_nuevo      = gas_pot
        pct_nuevo      = pct_pot
        obs = 'FALTAN DATOS- SE REPITE POTENCIAL'

    # — Regla 7 dentro del mismo flujo —
    if obs == 'VALIDAR INGENIERO' and pd.notna(potencia_nueva):
        if neta <= row['Potencial Neta'] <= n_pot:
            potencia_nueva = row['Potencial Neta']
            bruta_nueva    = row['Q sum deseada']
            gas_nuevo      = gas
            pct_nuevo      = pct_ctrl
            obs = 'SE CARGA NUEVO POTENCIAL- DIFERENCIA MAYOR 30%'

    # — Regla 8 dentro del mismo flujo —
    if pd.notna(potencia_nueva) and isinstance(potencia_nueva, (int,float)) and potencia_nueva < neta:
        potencia_nueva = neta
        bruta_nueva    = bruta
        gas_nuevo      = gas
        pct_nuevo      = pct_ctrl
        obs = 'POTENCIAL < CONTROL – SE TOMA COMO POTENCIAL EL CONTROL'

    return pd.Series([
        potencia_nueva,
        bruta_nueva,
        gas_nuevo,
        pct_nuevo,
        obs
    ], index=[
        'Potencial Neta Nuevo',
        'Potencial bruta',
        'Nuevo Potencia gas Nuevo',
        '% agua NUEVO potencial',
        'OBSERVACIONES'
    ])

## Aplico el cálculo en un único pase
df[['Potencial Neta Nuevo',
    'Potencial bruta',
    'Nuevo Potencia gas Nuevo',
    '% agua NUEVO potencial',
    'OBSERVACIONES']] = df.apply(calcular_nuevas_columnas, axis=1)

# Fecha de hoy y nombre de archivo
from datetime import datetime
hoy = datetime.now().strftime("%y/%d/%y")
output_file = "Potenciales a cargar.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    # 1) Pestaña con tus datos transformados
    df.to_excel(writer, sheet_name="Transformada", index=False)

    # 2) Pestaña 'Reporte'
    # — Preparamos cada serie, redondeando sólo los numéricos —
    pot_neto = pd.to_numeric(df["Potencial Neta Nuevo"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Potencial Neta Nuevo"])
    pot_gas  = pd.to_numeric(df["Nuevo Potencia gas Nuevo"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Nuevo Potencia gas Nuevo"])
    pot_bruta = pd.to_numeric(df["Potencial bruta"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Potencial bruta"])
    pct_agua = pd.to_numeric(df["% agua NUEVO potencial"], errors="coerce") \
                   .round(2) \
                   .fillna(df["% agua NUEVO potencial"])

    # Recalcular Pot Agua (numérico) y luego llenar vacíos
    bruta_num = pd.to_numeric(df["Potencial bruta"],      errors="coerce")
    neta_num  = pd.to_numeric(df["Potencial Neta Nuevo"], errors="coerce")
    pot_agua  = (bruta_num - neta_num).round(2).fillna('')

    reporte = pd.DataFrame({
        "Nombre Batería": df["Bateria"],
        "Nombre Pozo":    df["Pozo"],
        "Petroleo":       df["neta"],
        "Agua":           df["agua"],
        "Volumen total":  df["Bruta"],
        "Gas":            df["gas"],
        "Fecha Pot":      hoy,
        "Pot Neto":       pot_neto,
        "Pot Gas":        pot_gas,
        "Pot Bruto":      pot_bruta,
        "Pot Agua":       pot_agua,
        "%Agua":          pct_agua,
        "profundidad SE": df["Prof de Bba"],
        "Sumergencia":    df["Sumergencia"],
    })
    reporte.to_excel(writer, sheet_name="Reporte", index=False)
    
    # 3) potenciales a exportar
    CC = "CC"; PT = "PT"; DD = "DD"
    fecha_export = f"{datetime.now().day}/{datetime.now().month}/{datetime.now().year}"

    df3 = pd.DataFrame({
        "Nombre Pozo": df["Pozo"],
        "CC":           CC,
        "PT":           PT,
        "Fecha Pot":    fecha_export,
        "DD":           DD,
        "Pot Neto":     pot_neto,
        "Pot Agua":     pot_agua,
        "Pot Gas":      pot_gas,
        "Pot Bruto":    pot_bruta,
        "%Agua":        pct_agua,
    })
    df3.to_excel(writer,
                 sheet_name="potenciales a exportar",
                 index=False,
                 header=False)

print(f"Guardado en {output_file} con tres pestañas: Transformada, Reporte y potenciales a exportar.")


Guardado en Potenciales a cargar.xlsx con tres pestañas: Transformada, Reporte y potenciales a exportar.


In [7]:
#codigo correcto, este tenemos que usar para calculo de potenciales, sumado el de la consulta ( en el primer renglon)

import pandas as pd
import re
from datetime import datetime
import pandas as pd

# 0) Carga el Excel resultante de tu consulta
df = pd.read_excel("estructura_pozos.xlsx")

# 1) Renombrar columnas
rename_map = {
    'SYSDATE-VTOW_WELL_LAST_POTENTIAL_DET.MAX_EFF_DT':       'Días con potencial',
    'nivel_3':                                                'Activo',
    'nivel_5':                                                'Bateria',
    'prod_oil':                                               'neta',
    'prod_gas':                                               'gas',
    'free_wat_pct':                                           '%H20 control',
    'total_vol':                                              'Bruta',
    'pot_oil':                                                'Neta Potencial',
    'pot_gas':                                                'Gas Potencial',
    'VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT': 
                                                              'Bruta potencial',
    '(VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT*100)/DECODE((VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT),0,1,NULL,1,(VTOW_WELL_LAST_POTENTIAL_DET.POT_OIL+VTOW_WELL_LAST_POTENTIAL_DET.POT_WAT))': 
                                                              '%H20 potencial',
    'prof_ent_bomba':                                         'Prof de Bba',
    'sumergencia':                                            'Sumergencia',
    'tiponivel':                                              'Tipo-Nivel',
    'estado':                                                 'Estado',
    'nombre_corto':                                           'Pozo'
}

df = df.rename(columns=rename_map)

# 2) Quedarse solo con esas columnas
df = df[list(rename_map.values())]

# 2.1) Filtrar sólo registros en producción
df = df[df['Estado'] == 'Produciendo']

# 3) Columnas ya existentes calculadas previamente
df['agua'] = df['Bruta'] - df['neta']
df['agua potencial'] = df['Bruta potencial'] - df['Neta Potencial']

# —————————————————————————————————————————————
# 4) Nuevas columnas según tus puntos:
# 4.1) SUM DESEADA: igual a Sumergencia salvo que si >200 vale 200
df['SUM DESEADA'] = df['Sumergencia'].where(df['Sumergencia'] < 200, 200)

# 4.2) P estática: Prof de Bba dividido 10
df['P estática'] = df['Prof de Bba'] / 10

# 4.3) Q máximo: Q_max = Bruta / [1 – 0.2*(Sumergencia/Prof de Bba) – 0.8*(Sumergencia/Prof de Bba)^2]
ratio = df['Sumergencia'] / df['Prof de Bba']
df['Q máximo'] = df['Bruta'] / (1 - 0.2 * ratio - 0.8 * ratio**2)

# 4.4) Q sum deseada: Q_sum = Q_max * [ (1 – 0.2*((SUM DESEADA/10)/P estática)) – 0.8*((SUM DESEADA/10)/P estática)^2 ]
ratio_deseada = (df['SUM DESEADA'] / 10) / df['P estática']
df['Q sum deseada'] = df['Q máximo'] * ((1 - 0.2 * ratio_deseada) - 0.8 * ratio_deseada**2)

# 4.5) Potencial Neta: Q sum deseada * %H20 control
df['Potencial Neta'] = df['Q sum deseada'] * (100 - df['%H20 control']) / 100

# Nuevas columnas de diferencia
# 7) DIF NETA POTENCIAL VS CONTROL
df['DIF NETA POTENCIAL VS CONTROL'] = (df['Potencial Neta'] - df['neta']).round(2)

# 8) % DIF NETA POT VS CONTR (entero nullable para aceptar NaN)
pct = (df['Potencial Neta'] - df['neta']) / df['Potencial Neta'] * 100
df['% DIF NETA POT VS CONTR'] = pct.round(0).astype('Int64')

import numpy as np

# — Inicializo las nuevas columnas —
nuevas = [
    'Potencial Neta Nuevo',
    'Potencial bruta',
    'Nuevo Potencia gas Nuevo',
    '% agua NUEVO potencial',
    'OBSERVACIONES'
]
for col in nuevas:
    df[col] = pd.NA

import re

# — tras tus cálculos previos, antes del guardado —  


# … tu código de carga, rename_map y cálculos previos …

def calcular_nuevas_columnas(row):
    bateria     = str(row['Bateria'])
    sumergencia = row['Sumergencia']
    prof        = row['Prof de Bba']
    neta        = row['neta']
    bruta       = row['Bruta']
    n_pot       = row['Neta Potencial']
    bruta_pot   = row['Bruta potencial']
    gas         = row['gas']
    gas_pot     = row['Gas Potencial']
    pct_ctrl    = row['%H20 control']
    pct_pot     = row['%H20 potencial']
    dif_pct     = row['% DIF NETA POT VS CONTR']
    tipo_nivel  = str(row['Tipo-Nivel']).strip().lower()

    # 0) Swabing / Swabbing
    if re.search(r'swab(?:ing|bing)', bateria, flags=re.IGNORECASE):
        potencia_nueva = neta;  bruta_nueva = bruta
        gas_nuevo      = gas;   pct_nuevo = pct_ctrl
        obs = ''

    # 1) Estático o vacío
    elif tipo_nivel == 'estático' or tipo_nivel == '':
        potencia_nueva = n_pot;    bruta_nueva = bruta_pot
        gas_nuevo      = gas_pot;  pct_nuevo   = pct_pot
        obs = 'ULTIMO VALOR DE SUMERGENCIA ES ESTATICO – SE REPITE POTENCIAL'

    # 2) Sumergencia > Prof de Bba
    elif pd.notna(sumergencia) and sumergencia > prof:
        if neta < n_pot:
            potencia_nueva = n_pot;   bruta_nueva = bruta_pot
            gas_nuevo      = gas_pot; pct_nuevo   = pct_pot
            obs = 'VALOR DE SUMERGENCIA ERRONEO – SE REPITE POTENCIAL'
        else:
            potencia_nueva = neta;    bruta_nueva = bruta
            gas_nuevo      = gas;     pct_nuevo   = pct_ctrl
            obs = 'VALOR DE SUMERGENCIA ERRONEO – SE TOMA COMO POTENCIAL CONTROL'

    # 3) % DIF entre 0 y 30
    elif pd.notna(dif_pct) and 0 <= dif_pct <= 30:
        potencia_nueva = row['Potencial Neta']
        bruta_nueva    = row['Q sum deseada']
        gas_nuevo      = gas
        pct_nuevo      = pct_ctrl
        obs = 'SE CARGA NUEVO POTENCIAL'

    # 4) % DIF entre 30 y 65
    elif pd.notna(dif_pct) and 30 < dif_pct <= 65:
        potencia_nueva = 'DEFINIR INGENIERO'
        bruta_nueva    = 'DEFINIR INGENIERO'
        gas_nuevo      = 'DEFINIR INGENIERO'
        pct_nuevo      = 'DEFINIR INGENIERO'
        obs = 'VALIDAR INGENIERO'

    # 5) % DIF entre 65 y 100
    elif pd.notna(dif_pct) and 65 < dif_pct <= 100:
        potencia_nueva = row['Neta Potencial']
        bruta_nueva    = bruta_pot
        gas_nuevo      = gas_pot
        pct_nuevo      = pct_pot
        obs = 'VER IPR – SE REPITE POTENCIAL'

    # 6) Fallback
    else:
        potencia_nueva = row['Neta Potencial']
        bruta_nueva    = bruta_pot
        gas_nuevo      = gas_pot
        pct_nuevo      = pct_pot
        obs = 'FALTAN DATOS- SE REPITE POTENCIAL'

    # — Regla 7 dentro del mismo flujo —
    if obs == 'VALIDAR INGENIERO' and pd.notna(potencia_nueva):
        if neta <= row['Potencial Neta'] <= n_pot:
            potencia_nueva = row['Potencial Neta']
            bruta_nueva    = row['Q sum deseada']
            gas_nuevo      = gas
            pct_nuevo      = pct_ctrl
            obs = 'SE CARGA NUEVO POTENCIAL- DIFERENCIA MAYOR 30%'

    # — Regla 8 dentro del mismo flujo —
    if pd.notna(potencia_nueva) and isinstance(potencia_nueva, (int,float)) and potencia_nueva < neta:
        potencia_nueva = neta
        bruta_nueva    = bruta
        gas_nuevo      = gas
        pct_nuevo      = pct_ctrl
        obs = 'POTENCIAL < CONTROL – SE TOMA COMO POTENCIAL EL CONTROL'

    return pd.Series([
        potencia_nueva,
        bruta_nueva,
        gas_nuevo,
        pct_nuevo,
        obs
    ], index=[
        'Potencial Neta Nuevo',
        'Potencial bruta',
        'Nuevo Potencia gas Nuevo',
        '% agua NUEVO potencial',
        'OBSERVACIONES'
    ])

## Aplico el cálculo en un único pase
df[['Potencial Neta Nuevo',
    'Potencial bruta',
    'Nuevo Potencia gas Nuevo',
    '% agua NUEVO potencial',
    'OBSERVACIONES']] = df.apply(calcular_nuevas_columnas, axis=1)

# Fecha de hoy y nombre de archivo
from datetime import datetime
hoy = datetime.now().strftime("%y/%d/%y")
output_file = "Potenciales a cargar2.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    # 1) Pestaña con tus datos transformados
    df.to_excel(writer, sheet_name="Transformada", index=False)

    # 2) Pestaña 'Reporte'
    # — Preparamos cada serie, redondeando sólo los numéricos —
    pot_neto = pd.to_numeric(df["Potencial Neta Nuevo"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Potencial Neta Nuevo"])
    pot_gas  = pd.to_numeric(df["Nuevo Potencia gas Nuevo"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Nuevo Potencia gas Nuevo"])
    pot_bruta = pd.to_numeric(df["Potencial bruta"], errors="coerce") \
                   .round(2) \
                   .fillna(df["Potencial bruta"])
    pct_agua = pd.to_numeric(df["% agua NUEVO potencial"], errors="coerce") \
                   .round(2) \
                   .fillna(df["% agua NUEVO potencial"])

    # Recalcular Pot Agua (numérico) y luego llenar vacíos
    bruta_num = pd.to_numeric(df["Potencial bruta"],      errors="coerce")
    neta_num  = pd.to_numeric(df["Potencial Neta Nuevo"], errors="coerce")
    pot_agua  = (bruta_num - neta_num).round(2).fillna('')

    reporte = pd.DataFrame({
        "Nombre Batería": df["Bateria"],
        "Nombre Pozo":    df["Pozo"],
        "Petroleo":       df["neta"],
        "Agua":           df["agua"],
        "Volumen total":  df["Bruta"],
        "Gas":            df["gas"],
        "Fecha Pot":      hoy,
        "Pot Neto":       pot_neto,
        "Pot Gas":        pot_gas,
        "Pot Bruto":      pot_bruta,
        "Pot Agua":       pot_agua,
        "%Agua":          pct_agua,
        "profundidad SE": df["Prof de Bba"],
        "Sumergencia":    df["Sumergencia"],
    })
    
    # — Asegurarnos de que los Pot* sean numéricos (los no convertibles pasarán a NaN) —
    for col in ['Pot Neto', 'Pot Agua', 'Pot Bruto', 'Pot Gas']:
        reporte[col] = pd.to_numeric(reporte[col], errors='coerce')
    
    mask = (
        (reporte['Petroleo']      < 0.01) &
        (reporte['Agua']          < 0.01) &
        (reporte['Volumen total'] < 0.01) &
        (reporte['Gas']           < 0.01)
    )
    reporte.loc[mask, 'Pot Neto']      = 0.01
    reporte.loc[mask, 'Pot Bruto'] = 0.01
    reporte.loc[mask, '%Agua'] = 100
    
    
    reporte.to_excel(writer, sheet_name="Reporte", index=False)
    
    # … tras aplicar df.apply(calcular_nuevas_columnas) …

    # 1) Defino las columnas de potencial donde buscar 'DEFINIR INGENIERO'
    pot_cols = [
        "Potencial Neta Nuevo",
        "Potencial bruta",
        "Nuevo Potencia gas Nuevo",
        "% agua NUEVO potencial"
    ]

    # 2) Construyo máscara: keep sólo filas donde NINGUNA de esas columnas sea 'DEFINIR INGENIERO'
    mask = ~df[pot_cols].eq("DEFINIR INGENIERO").any(axis=1)
    df_export = df[mask].copy()

    # 3) Variables fijas de exportación
    CC = "CC"
    PT = "PT"
    DD = "DD"
    fecha_export = f"{datetime.now().day}/{datetime.now().month}/{datetime.now().year}"

    # 4) Recalculo series numéricas sobre df_export (igual que antes)
    pot_neto  = pd.to_numeric(df_export["Potencial Neta Nuevo"], errors="coerce") .round(2).fillna(df_export["Potencial Neta Nuevo"])
    pot_gas   = pd.to_numeric(df_export["Nuevo Potencia gas Nuevo"], errors="coerce") .round(2).fillna(df_export["Nuevo Potencia gas Nuevo"])
    pot_bruta = pd.to_numeric(df_export["Potencial bruta"], errors="coerce")        .round(2).fillna(df_export["Potencial bruta"])
    pct_agua  = pd.to_numeric(df_export["% agua NUEVO potencial"], errors="coerce") .round(2).fillna(df_export["% agua NUEVO potencial"])
    bruta_num = pd.to_numeric(df_export["Potencial bruta"], errors="coerce")
    neta_num  = pd.to_numeric(df_export["Potencial Neta Nuevo"], errors="coerce")
    pot_agua  = (bruta_num - neta_num).round(2).fillna('')

    # 5) Armo el DataFrame de exportación
    df3 = pd.DataFrame({
        "Nombre Pozo": df_export["Pozo"],
        "CC":           CC,
        "PT":           PT,
        "Fecha Pot":    fecha_export,
        "DD":           DD,
        "Pot Neto":     pot_neto,
        "Pot Agua":     pot_agua,
        "Pot Gas":      pot_gas,
        "Pot Bruto":    pot_bruta,
        "%Agua":        pct_agua,
    })

    # 6) Inserto la columna 'Activo' en la posición K (índice 10)
    df3.insert(10, "Activo", df_export["Activo"])

    
    # — Forzamos mínimos en potenciales también en df3 —
    # Nos aseguramos de que sean numéricos
    for c in ['Pot Neto','Pot Agua','Pot Bruto','Pot Gas','%Agua']:
        df3[c] = pd.to_numeric(df3[c], errors='coerce')

    mask3 = (
        (df3['Pot Neto']   < 0.01) &
        (df3['Pot Agua']   < 0.01) &
        (df3['Pot Bruto']  < 0.01) &
        (df3['Pot Gas']    < 0.01)
    )
    df3.loc[mask3, 'Pot Neto']  = 0.01
    df3.loc[mask3, 'Pot Bruto'] = 0.01
    df3.loc[mask3, '%Agua']     = 100
    
    # 7) Y por último vuelco sin encabezados
    df3.to_excel(writer,
                 sheet_name="potenciales a exportar",
                 index=False,
                 header=False)

    # Mapeo de nombre de sheet → valor de "Activo" a filtrar
    sheets = {
        "Pot LH": "Las Heras CG - Canadon Escondida",
        "Pot LP": "Los Perales",
        "Pot CS": "Seco Leon - Pico Truncado",
        "Pot Eg": "El Guadal"
    }

    for sheet_name, activo_val in sheets.items():
        # filtrar sólo filas con ese 'Activo'
        df_sheet = df3[df3["Activo"] == activo_val].copy()
        # quedarnos sólo con las primeras 10 columnas (A–J)
        df_sheet = df_sheet.iloc[:, :10]
        # volcar sin índices ni encabezados
        df_sheet.to_excel(writer,
                          sheet_name=sheet_name,
                          index=False,
                          header=False)

print(f"Guardado en {output_file} con tres pestañas: Transformada, Reporte y potenciales a exportar.")

Guardado en Potenciales a cargar2.xlsx con tres pestañas: Transformada, Reporte y potenciales a exportar.


In [8]:
csv_info = [
    ("Las Heras CG - Canadon Escondida", "potencialesLH.csv"),
    ("Los Perales",                 "potencialesLp.csv"),
    ("Seco Leon - Pico Truncado",   "potencialesCs.csv"),
    ("El Guadal",                   "potencialesEg.csv"),
]

for activo_val, filename in csv_info:
    # Filtrar solo registros de este 'Activo'
    df_csv = df3[df3["Activo"] == activo_val].copy()
    # Conservar solo columnas A–J
    df_csv = df_csv.iloc[:, :10]
    # Guardar como CSV delimitado por comas, sin índice ni header
    df_csv.to_csv(filename, index=False, header=False)

print("CSV generados: ", [fn for _, fn in csv_info])

CSV generados:  ['potencialesLH.csv', 'potencialesLp.csv', 'potencialesCs.csv', 'potencialesEg.csv']
